In [7]:
import requests
import boto3
import json
from decimal import Decimal

In [8]:
#cnpjs que serão utilizados em nosso teste
cnpjs = ['04203570000130','76726884005944','79080602000407','06318618001829','34422807000134',
         '16617588000167','09169438000172','30998254000101','11318464000167','56012628000161']

In [9]:
#fazendo conexão com o banco nosql dynamo para salvarmos os retornos da api
dynamodb = boto3.resource('dynamodb',
                          region_name='us-east-1',
                          aws_access_key_id='#######',
                          aws_secret_access_key='######')

In [10]:
#tabela que será usada para salvar os retornos
table = dynamodb.Table('fc_empresa_teste_bigdata')

In [11]:
#fazendo consulta a api da bigdata
endpoint = 'https://bigboost.bigdatacorp.com.br/companies'
headers = {'Content-Type': 'application/json; charset=UTF-8'}
token = 'AB33E58C-2537-47ED-9F1C-8D6232E45542'
datasets = ['basic_data', 'addresses_extended', 'phones_extended', 'emails_extended', 'relationships', 
            'activity_indicators','ondemand_query_pesquisaprotesto', 'processes']

for cnpj in cnpjs:
    print('iniciando cnpj: ', cnpj)
    for dataset in datasets:
        print('----> pegando dados do dataset: ', dataset)  
        
        if dataset == 'ondemand_query_pesquisaprotesto':
            endpoint = "https://bigboost.bigdatacorp.com.br/companies?Datasets=ondemand_query_pesquisaprotesto&q=doc{"+cnpj+"}&AccessToken="+token
            response = requests.get(endpoint, headers=headers)
        elif dataset == 'relationships':
            request = {
                "Datasets": 'relationships.filter(relationshiptype=[QSA,Ownership,RepresentanteLegal])',
                "q": "doc{"+cnpj+"}",
                "AccessToken": token
            }
            response = requests.post(endpoint, headers=headers, data=json.dumps(request))
            print('----> resposta da api:', response.json())
        elif dataset == 'processes':
            request = {
                "Datasets": 'processes.filter(party_type=[defendant],courtlevel=[1],mainsubject=[DESPEJO, COBRANÇA DE ALUGUEIS,COBRANÇA DE ALUGUEL,COBRANÇA DE ALUGUEIS - SEM DESPEJO,COBRANÇA DE ALUGUEL - SEM DESPEJO,DESPEJO CUMULADO COM COBRANÇA,DESPEJO POR FALTA DE PAGAMENTO,DESPEJO POR INFRAÇÃO CONTRATUAL,LOCACAO DE IMOVEL - INADIMPLEMENTO,INADIMPLEMENTO DE ALUGUEL,INADIMPLEMENTO DE ALUGUEIS,INADIMPLÊNCIA DE ALUGUEL,INADIMPLÊNCIA DE ALUGUEIS])',
                "q": "doc{"+cnpj+"}",
                "AccessToken": token
            }
            response = requests.post(endpoint, headers=headers, data=json.dumps(request))
            print('----> resposta da api:', response.json())
        else:
            request = {
                "Datasets": dataset,
                "q": "doc{"+cnpj+"}",
                "AccessToken": token
            }
            response = requests.post(endpoint, headers=headers, data=json.dumps(request))
        table.put_item(
            Item={
                'document': cnpj,
                'dataset': dataset,
                'data': json.loads(json.dumps(response.json()), parse_float=Decimal)
                }
        )
    print('finalizando cnpj: ', cnpj)
    print('------------------------')

iniciando cnpj:  04203570000130
----> pegando dados do dataset:  processes
----> resposta da api: {'Result': [{'MatchKeys': 'doc{04203570000130}', 'Lawsuits': {'Lawsuits': [], 'TotalLawsuits': 19, 'TotalLawsuitsAsAuthor': 15, 'TotalLawsuitsAsDefendant': 4, 'TotalLawsuitsAsOther': 0, 'Last30DaysLawsuits': 0, 'Last90DaysLawsuits': 0, 'Last180DaysLawsuits': 0, 'Last365DaysLawsuits': 0}}], 'QueryId': 'ba570ce2-666e-40c7-8d4b-4b4a7b3377eb', 'ElapsedMilliseconds': 1414.0, 'Status': {'processes': [{'Code': 0, 'Message': 'OK'}]}}
finalizando cnpj:  04203570000130
------------------------
iniciando cnpj:  09169438000172
----> pegando dados do dataset:  processes
----> resposta da api: {'Result': [{'MatchKeys': 'doc{09169438000172}', 'Lawsuits': {'Lawsuits': [], 'TotalLawsuits': 588, 'TotalLawsuitsAsAuthor': 136, 'TotalLawsuitsAsDefendant': 427, 'TotalLawsuitsAsOther': 25, 'Last30DaysLawsuits': 0, 'Last90DaysLawsuits': 0, 'Last180DaysLawsuits': 0, 'Last365DaysLawsuits': 0, 'NextPageId': 'eyJ0eXA